## Notbook D: Adding motion to the simulation

#### Prerequisites:
- segmentation and motion fields
- acquisition template rawdata

#### Goals:
- Simulating a steady-state sequence including motion

#### Content overview: 
- generating a surrogate signal
- combining surrogate signal and motion model
- adding both cardiac and respiratory motion to the simulation
- storing the ground truth motion information

In [ ]:
# first we need to set up the simulation again

from pathlib import Path
import os
import numpy as np 
import sirf.Gadgetron as pMR
import sirf.Reg as pReg
import sirf.DynamicSimulation as pDS
import auxiliary_functions as aux

root_path = Path(os.getenv("SIRF_INSTALL_PATH"))
root_path = root_path / "share/SIRF-3.1/Simulation/"
fpath_input = root_path / "Input"


In [ ]:
#these steps just repeat the previous notebook
# set up simulation as we know it from before
fname_xml = fpath_input / "XCAT_TissueParameters_XML.xml"
fname_segmentation = fpath_input / "segmentation.nii"

segmentation = pReg.NiftiImageData3D(str(fname_segmentation))
simulation = pDS.MRDynamicSimulation(segmentation, str(fname_xml))

# fname_contrast_template = fpath_input / "contrast_template.h5"
# contrast_template = pMR.AcquisitionData(str(fname_contrast_template))
# contrast_template = pMR.preprocess_acquisition_data(contrast_template)

fname_acquisition_template = fpath_input / "acquisition_template.h5"
acquisition_template = pMR.AcquisitionData(str(fname_acquisition_template))

num_acq_subset = 256
subset = np.arange(num_acq_subset)
acquisition_template = acquisition_template.get_subset(subset)

# simulation.set_contrast_template_data(contrast_template)
simulation.set_contrast_template_data(acquisition_template)

# to activate a golden-angle 2D encoding model we only need to set the trajectory
acquisition_template = pMR.set_goldenangle2D_trajectory(acquisition_template)
simulation.set_acquisition_template_data(acquisition_template)

# compute the CSM from the raw data itself
csm = pMR.CoilSensitivityData()
csm.smoothness = 50
csm.calculate(acquisition_template)

# you can potentially keep 
# however, then you need to set the trajectory above that the rawdata used for sampling.
csm_datatype = csm.as_array().dtype
csm_shape = csm.as_array().shape
unity_csm = np.ones(shape=csm_shape, dtype=csm_datatype)
csm.fill(unity_csm)

simulation.set_csm(csm)


# we set up our transformation to get a 4-chamber view
offset_x_mm = 0
offset_y_mm = 0
offset_z_mm = -4.5
rotation_angles_deg = [0,0,0]
translation = np.array([offset_x_mm, offset_y_mm, offset_z_mm])
euler_angles_deg = np.array(rotation_angles_deg)

offset_trafo = pReg.AffineTransformation(translation, euler_angles_deg)
simulation.set_offset_trafo(offset_trafo)

# without external signal you can set an SNR
SNR = 10
SNR_label = 13

simulation.set_snr(SNR)
simulation.set_snr_label(SNR_label)
simulation.simulate_data()

# now we simulate and 
fname_output = root_path / "Output/output_c_simulate_motion_static.h5"
if not fname_output.parent.is_dir():
    fname_output.parent.mkdir(parents=True, exist_ok=True)

simulation.write_simulation_results(str(fname_output))


In [ ]:
simulated_file = pMR.AcquisitionData(str(fname_output))
recon_resp = aux.iterative_reconstruct_data(simulated_file)

recon_resp = pReg.NiftiImageData3D(recon_resp)
recon_resp.write('/media/sf_CCPPETMR/tmp_c_stat.nii')

### Motion Dynamics
To add motion to the simulation we need to construct an object called a MotionDynamic.
They consist of two things:
- a surrogate signal
- a set of displacement vector fields

Furthermore you need to define 
- how many motion states you want to simulate.
- where the ground truth should be stored to.

In [ ]:
import matplotlib.pyplot as plt

Nt = 10000
t0_s = 0
tmax_s = 60* 1 
f_Hz_resp = 0.25

t_resp, sig_resp = aux.get_normed_sinus_signal(t0_s, tmax_s, Nt, f_Hz_resp)

plt.figure()
plt.plot(t_resp, sig_resp)
plt.xlabel("time(sec)")
plt.ylabel("sig (a.u.)")
plt.title("Respiration signal")
plt.show()

In [ ]:
# configure the motion
num_motion_states = 3

# RESP
num_sim_resp_states = num_motion_states
resp_motion = pDS.MRMotionDynamic( num_sim_resp_states )
resp_motion.set_dynamic_signal(t_resp, sig_resp)
resp_motion.set_cyclicality(False)
resp_motion.set_groundtruth_folder_prefix(str(root_path / "Output/gt_resp/"))

aux.set_motionfields_from_path(resp_motion, str(fpath_input / 'mvfs_resp/'))
simulation.add_motion_dynamic(resp_motion)

In [ ]:
simulation.simulate_data()

# now we simulate and 
fname_output = root_path / "Output/output_c_simulate_motion_breathing.h5"
if not fname_output.parent.is_dir():
    fname_output.parent.mkdir(parents=True, exist_ok=True)

simulation.write_simulation_results(str(fname_output))

In [ ]:

simulated_file = pMR.AcquisitionData(str(fname_output))

recon_resp = aux.iterative_reconstruct_data(simulated_file)
recon_resp = pReg.NiftiImageData3D(recon_resp)
recon_resp.write('/media/sf_CCPPETMR/tmp_c_resp.nii')

In [ ]:
Nt = 10000
t0_s = 0
tmax_s = 60*0.25 
f_Hz_card = 1.25

t_card, sig_card = aux.get_normed_sawtooth_signal(t0_s, tmax_s, Nt, f_Hz_card)

plt.figure()
plt.plot(t_card, sig_card)
plt.xlabel("time(sec)")
plt.ylabel("sig (a.u.)")
plt.title("ECG")
plt.show()

In [ ]:
# CARD
num_sim_card_states = num_motion_states
card_motion = pDS.MRMotionDynamic( num_sim_card_states )
card_motion.set_dynamic_signal(t_card, sig_card)
card_motion.set_cyclicality(False)
card_motion.set_groundtruth_folder_prefix(str(root_path / "Output/gt_card/"))

aux.set_motionfields_from_path(card_motion, str(fpath_input / 'mvfs_card/'))
simulation.add_motion_dynamic(card_motion)



In [ ]:
simulation.simulate_data()

# now we simulate and 
fname_output = root_path / "Output/output_c_simulate_motion_cardio_respiratory.h5"
if not fname_output.parent.is_dir():
    fname_output.parent.mkdir(parents=True, exist_ok=True)

simulation.write_simulation_results(str(fname_output))

In [ ]:

simulated_file = pMR.AcquisitionData(str(fname_output))

recon_card = aux.iterative_reconstruct_data(simulated_file)
recon_card = pReg.NiftiImageData3D(recon_resp)
recon_card.write('/media/sf_CCPPETMR/tmp_c_cardresp.nii')